In [14]:
from glob import glob
from datasets import Dataset, concatenate_datasets
import numpy as np
import pandas as pd
from pprint import pprint
import os
import json

In [6]:
with open('./data/end_conversation.txt', 'r') as f:
    end_conversations = [x.lower() for x in f.read().split('\n')]

In [17]:
len(end_conversations)

700

In [15]:
oasst = Dataset.from_json('./clean_data/oasst.jsonl').shuffle(seed=42).select(range(150))
quora = Dataset.from_json('./clean_data/quora.jsonl').shuffle(seed=42).select(range(150))
wivi = Dataset.from_json('./clean_data/wivi.jsonl').shuffle(seed=42).select(range(100))
vi_ultra = Dataset.from_parquet('./clean_data/vi_ultra.parquet').shuffle(seed=42).select(range(300))
final_ds = concatenate_datasets([oasst, quora, wivi, vi_ultra])

In [21]:
final_ds[0]

{'messages': [{'content': 'Tóm tắt văn bản được cung cấp và biến nó thành thẻ. Mỗi thẻ nên chứa một câu hỏi và một câu trả lời ngắn.\n\nPhương pháp lập trình động tương tự như chia để trị trong việc chia nhỏ vấn đề thành các vấn đề con nhỏ hơn và nhỏ hơn nữa. Nhưng không giống như chia để trị, những vấn đề phụ này không được giải quyết một cách độc lập. Thay vào đó, kết quả của các bài toán con nhỏ hơn này được ghi nhớ và sử dụng cho các bài toán con tương tự hoặc chồng chéo.\n\nLập trình động được sử dụng khi chúng ta gặp vấn đề, có thể chia thành các vấn đề con tương tự để có thể sử dụng lại kết quả của chúng. Hầu hết, các thuật toán này được sử dụng để tối ưu hóa. Trước khi giải bài toán con đang làm, thuật toán động sẽ thử kiểm tra kết quả của bài toán con đã giải trước đó. Các giải pháp của các vấn đề con được kết hợp để đạt được giải pháp tốt nhất.\n\nVì vậy, chúng ta có thể nói rằng -\n\nVấn đề sẽ có thể được chia thành vấn đề con chồng chéo nhỏ hơn.\n\nMột giải pháp tối ưu có t

In [30]:
TEMPLATE = """Dưới đây là lịch sử hội thoại cho đến nay, và một câu hỏi mởi đặt ra bởi người dùng. Dựa vào lịch sử hội thoại và câu hỏi mới, hãy viết lại một câu hỏi hoàn chỉnh chứa thông tin ngữ cảnh của văn bản.
Lịch sử trò chuyện:
{history_text}
Câu hỏi mới từ #User: {new_question}"""
i = 0
def mapper(x):
    global i
    messages = [t['content'] for t in x['messages'] if t['role'] == 'user']
    messages = messages[:np.random.randint(1, 3)]
    history_text = '\n'.join([f'#User: {x}' for x in messages])
    new_question = end_conversations[i]
    i += 1
    prompt = TEMPLATE.format(history_text=history_text, new_question=new_question)
    return {
        'messages': [
            {'role':'user', 'content': prompt},
            {'role':'assistant', 'content': new_question}
        ]
    }

temp_final = final_ds.map(mapper).select_columns(['messages'])

In [55]:
a = temp_final.shuffle()[0]
print(a['messages'][0]['content'])
print(a['messages'][1]['content'])

Dưới đây là lịch sử hội thoại cho đến nay, và một câu hỏi mởi đặt ra bởi người dùng. Dựa vào lịch sử hội thoại và câu hỏi mới, hãy viết lại một câu hỏi hoàn chỉnh chứa thông tin ngữ cảnh của văn bản.
Lịch sử trò chuyện:
#User: Ghế ăn Emily sẽ phù hợp với bất kỳ bàn ăn nào đang tìm kiếm một chiếc ghế thiết kế hiện đại.
Ghế được bọc nệm bằng lụa màu mù tạt, màu hải quân hoặc màu onyx. Chân màu đen.
Bạn có thể gợi ý những chiếc ghế ăn khác nào có thiết kế hiện đại giống như ghế ăn Emily không?
#User: Đó là một danh sách tuyệt vời về ghế ăn! Bạn có thể giới thiệu một chiếc ghế tương tự như ghế ăn Emily nhưng có tùy chọn màu sắc khác không?
Câu hỏi mới từ #User: phải chiếc gì
phải chiếc gì


In [57]:
temp_final.to_json('./rewrite_end_conversation.jsonl', orient='records', lines=True, force_ascii=False)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

748713